In [ ]:
# 해야할 일
# 1)중요도가 낮은 Feature라 제거하였음을 나타낼 방법 -> LGBM 사용?

## 1) Data Cleansing

### Import

In [1]:
import pandas as pd
import warnings ; warnings.filterwarnings('ignore')

### Read Data

In [3]:
train = pd.read_csv('./data/raw/train.csv', parse_dates=['base_date'])
test = pd.read_csv('./data/raw/test.csv', parse_dates=['base_date'])

### Data Cleansing

#### $\bullet$ [Drop] categorical variable
특정값이 많은 변수는 데이터를 잘 설명하지 못한다. 변수 내 범주빈도를 구한 뒤 가장 큰 범주빈도가 0.95 이상이면 제거한다.

In [4]:
category = ['road_rating', 'road_name', 'multi_linked', 'connect_code', 'maximum_speed_limit', 'vehicle_restricted', 'weight_restricted',
            'height_restricted', 'road_type', 'start_node_name','start_turn_restricted', 'end_node_name','end_turn_restricted']

In [5]:
category_dense = train[category].apply(lambda x: x.value_counts(normalize=True).iloc[0])

print(f'삭제되는 열은 {category_dense[category_dense > 0.95].index.values} 입니다.')
train.drop(category_dense[category_dense > 0.95].index, axis=1, inplace=True)
test.drop(category_dense[category_dense > 0.95].index, axis=1, inplace=True)

삭제되는 열은 ['multi_linked' 'connect_code' 'vehicle_restricted' 'height_restricted'] 입니다.


#### $\bullet$ [Labeling] node name

In [7]:
label_df = pd.concat([train['start_node_name'].rename('node_name'), train['end_node_name'].rename('node_name')]).drop_duplicates()\
           .reset_index(drop=True).reset_index().rename(columns={'index':'node_label'}).set_index('node_name')

for label in pd.concat([test['start_node_name'].rename('node_name'), test['end_node_name'].rename('node_name')]).drop_duplicates():
    if label not in label_df.index:
        label_df.loc[label] = label_df['node_label'].max() + 1

In [8]:
train['start_node_label'] = pd.merge(train[['id','start_node_name']], 
                                     label_df.reset_index().rename(columns={'node_name':'start_node_name'}), 
                                     on='start_node_name', how='left')['node_label']
train['end_node_label'] = pd.merge(train[['id','end_node_name']], 
                                   label_df.reset_index().rename(columns={'node_name':'end_node_name'}), 
                                   on='end_node_name', how='left')['node_label']

In [9]:
test['start_node_label'] = pd.merge(test[['id','start_node_name']], 
                                     label_df.reset_index().rename(columns={'node_name':'start_node_name'}), 
                                     on='start_node_name', how='left')['node_label']
test['end_node_label'] = pd.merge(test[['id','end_node_name']], 
                                   label_df.reset_index().rename(columns={'node_name':'end_node_name'}), 
                                   on='end_node_name', how='left')['node_label']

In [10]:
# labeling한 node_name은 제거한다.
del train['start_node_name'], train['end_node_name']
del test['start_node_name'], test['end_node_name']

#### $\bullet$ [Labeling] edge
- start_node와 end_node 사이를 edge라 칭하고 이를 labeling한 것이다.

In [11]:
label_df = train[['start_node_label','end_node_label']].drop_duplicates()\
           .reset_index(drop=True).reset_index().rename(columns={'index':'edge_label'}).set_index(['start_node_label','end_node_label'])

for label in test[['start_node_label','end_node_label']].drop_duplicates().values:
    if tuple(label) not in label_df.index:
        label_df.loc[tuple(label), 'edge_label'] = label_df['edge_label'].max()+1

In [12]:
train['edge_label'] = pd.merge(train[['id','start_node_label','end_node_label']], label_df.reset_index(), 
                               on=['start_node_label','end_node_label'], how='left')['edge_label']
test['edge_label'] = pd.merge(test[['id','start_node_label','end_node_label']], label_df.reset_index(), 
                               on=['start_node_label','end_node_label'], how='left')['edge_label']

#### $\bullet$ [Labeling] coordinate
- 출발지점과 도착지점의 좌표를 labeling한 것이다. 이때, 반대방향, 다시 말해 출발지점과 도착지점이 반대되는 label은 음수로 처리한다.

In [13]:
label_df = train[['start_latitude','start_longitude','end_latitude','end_longitude']].drop_duplicates()\
           .reset_index(drop=True).reset_index().rename(columns={'index':'coordinate_label'})\
           .set_index(['start_latitude','start_longitude','end_latitude','end_longitude'])

for label in test[['start_latitude','start_longitude','end_latitude','end_longitude']].drop_duplicates().values:
    if tuple(label) not in label_df.index:
        label_df.loc[tuple(label), 'coordinate_label'] = label_df['coordinate_label'].max()+1  

In [14]:
label_df = label_df.reset_index()

# 방향이 반대인 것은 Minus로 labeling한다.
for IDX, SLAT, SLON, ELAT, ELON, COOR in label_df.itertuples():
    CHANGE = label_df.loc[IDX:].query('(start_latitude==@ELAT) & (start_longitude==@ELON) &\
                                       (end_latitude==@SLAT) & (end_longitude==@SLON)').index
    if len(CHANGE)==1:
        label_df.iloc[CHANGE, -1] = -1 * COOR

In [15]:
train['coor_label'] = pd.merge(train[['id','start_latitude','start_longitude','end_latitude','end_longitude']], label_df, 
                               on=['start_latitude','start_longitude','end_latitude','end_longitude'], how='left')['coordinate_label']
test['coor_label'] = pd.merge(test[['id','start_latitude','start_longitude','end_latitude','end_longitude']], label_df, 
                              on=['start_latitude','start_longitude','end_latitude','end_longitude'], how='left')['coordinate_label']

#### $\bullet$ [Drop] Unuseful Data
- 분석에 용이하지 않은 데이터는 메모리 효율을 고려해 제거하는 것이 좋다. Feature Engineering에 사용하지 않을 변수는 제거한다.

In [16]:
unuse = ['lane_count','weight_restricted']
train.drop(unuse, axis=1, inplace=True)
test.drop(unuse, axis=1, inplace=True)

### Save Data

In [17]:
train.to_csv('../data/clean/train.csv', index=False)
test.to_csv('../data/clean/test.csv', index=False)